# 정칙화
- 다항회귀 시 차수가 높아 질 수록 과대적합이 발생하여 예측 성능이 저하되는 문제가 발생
- 과대적합을 방지를 위해 가중치 크기 제어
- 기존의 비용 함수에 규제(Penalty) 항을 추가하여 가중치의 크기를 제어 가능
- scikit-learn의 linear_model에 구현되어 있는 Ridge, Lasso, ElasticNet을 사용하여 규제 모델 사용가능

- 규제 강도(Alpha)
    - Alpha의 값을 아주 작게 하거나 0으로 지정하는 경우 가중치에 대한 규제가 약해지며 과대적합 발생
    - Alpha의 값을 너무 크게 지정하는 경우 가중치에 대한 규제가 강해지며 너무 모델이 단순하여 과소적합 발생

In [1]:
# 환경 설정
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split  
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression

In [2]:
# 회귀 데이터 가져오기
df = pd.read_csv('/mnt/elice/dataset/diabetes.csv')
df

,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6,target
0,0.038076,0.050680,0.061696,0.021872,-0.044223,-0.034821,-0.043401,-0.002592,0.019908,-0.017646,151.0
1,-0.001882,-0.044642,-0.051474,-0.026328,-0.008449,-0.019163,0.074412,-0.039493,-0.068330,-0.092204,75.0
2,0.085299,0.050680,0.044451,-0.005671,-0.045599,-0.034194,-0.032356,-0.002592,0.002864,-0.025930,141.0
3,-0.089063,-0.044642,-0.011595,-0.036656,0.012191,0.024991,-0.036038,0.034309,0.022692,-0.009362,206.0
4,0.005383,-0.044642,-0.036385,0.021872,0.003935,0.015596,0.008142,-0.002592,-0.031991,-0.046641,135.0
...,...,...,...,...,...,...,...,...,...,...,...
437,0.041708,0.050680,0.019662,0.059744,-0.005697,-0.002566,-0.028674,-0.002592,0.031193,0.007207,178.0
438,-0.005515,0.050680,-0.015906,-0.067642,0.049341,0.079165,-0.028674,0.034309,-0.018118,0.044485,104.0
439,0.041708,0.050680,-0.015906,0.017282,-0.037344,-0.013840,-0.024993,-0.011080,-0.046879,0.015491,132.0
440,-0.045472,-0.044642,0.039062,0.001215,0.016318,0.015283,-0.028674,0.026560,0.044528,-0.025930,220.0


In [3]:
# 데이터 분할하기
X = df.iloc[:, :-1]
Y = df.iloc[:, -1]
X.head()

,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6
0,0.038076,0.050680,0.061696,0.021872,-0.044223,-0.034821,-0.043401,-0.002592,0.019908,-0.017646
1,-0.001882,-0.044642,-0.051474,-0.026328,-0.008449,-0.019163,0.074412,-0.039493,-0.068330,-0.092204
2,0.085299,0.050680,0.044451,-0.005671,-0.045599,-0.034194,-0.032356,-0.002592,0.002864,-0.025930
3,-0.089063,-0.044642,-0.011595,-0.036656,0.012191,0.024991,-0.036038,0.034309,0.022692,-0.009362
4,0.005383,-0.044642,-0.036385,0.021872,0.003935,0.015596,0.008142,-0.002592,-0.031991,-0.046641


In [4]:
# 다항 변환 적용
P = PolynomialFeatures(degree=3, include_bias=False)
X_Poly = P.fit_transform(X)
X.shape, X_Poly.shape

((442, 10), (442, 285))

In [5]:
# 학습용과 평가용으로 데이터 분할
x_train, x_test, y_train, y_test = train_test_split(X_Poly, Y, random_state=0)

In [6]:
# 모델 생성
model = LinearRegression() 

In [7]:
# 모델 학습
model.fit(x_train, y_train) # 학습용 데이터만 사용

LinearRegression()

In [8]:
# 모델 평가 (R Squared)
print('학습 데이터 성능 :', model.score(x_train, y_train))
print('평가 데이터 성능 :', model.score(x_test, y_test))
# 학습 데이터 성능이 0.91로 굉장히 높지만, 평가 데이터 성능은 최악!
# 과대적합 발생했다.

학습 데이터 성능 : 0.9188658585737726
평가 데이터 성능 : -21.339849667350748


# Ridge
- 기존의 선형 회귀 식에 규제항(L2 Regularization)이 적용된 모델
- MSE가 최소가 되게 하는 가중치와 편향을 찾는 동시에 가중치들의 제곱 합이 최소가 되도록 적절한 가중치와 편향을 찾음
- L2 규제의 효과로 0이 되진 않지만 모델 복잡도를 제어할 수 있음
- 특성들이 출력에 미치는 영향력이 줄어듦(현재 특성들에 덜 의존)

In [9]:
# 규제 모델 생성 (LinearRegression이랑 똑같은데 항을 추가한 것)
from sklearn.linear_model import Ridge
model = Ridge(alpha=1) 

In [11]:
# 모델 학습
model.fit(x_train, y_train) # 학습용 데이터만 사용

Ridge(alpha=1)

In [12]:
# 모델 평가 (R Squared)
print('학습 데이터 성능 :', model.score(x_train, y_train))
print('평가 데이터 성능 :', model.score(x_test, y_test))
# 규제만 했는데도 평가 성능이 상당히 좋아짐
# 대신 둘 다 성능이 낮아서 과소적합 문제 발생! -> 규제가 너무 강했다.

학습 데이터 성능 : 0.46333427173027797
평가 데이터 성능 : 0.3571528311583685


In [13]:
# 규제 강도 완화 
model = Ridge(alpha=0.1) 
model.fit(x_train, y_train) 
print('학습 데이터 성능 :', model.score(x_train, y_train))
print('평가 데이터 성능 :', model.score(x_test, y_test))
# 개선됨!

학습 데이터 성능 : 0.5547492878993845
평가 데이터 성능 : 0.3715849639261284


# Lasso
- 기존의 선형 회귀 식에 규제항(L1 Regularization)이 적용된 모델
- MSE가 최소가 되게 하는 가중치와 편향을 찾는 동시에 가중치들의 절댓값 합이 최소가 되도록 적절한 가중치와 편향을 찾음
- L1 규제의 효과로 어떤 특성들은 0이 되어서 모델을 만들 때 사용되지 않음
    - 특성 선택(feature selection) 효과가 있음
- 모델에서 가장 중요한 특성이 무엇인지 알게 되어 모델 해석력이 좋아짐


In [14]:
# 규제 모델 생성
from sklearn.linear_model import Lasso
model = Lasso(alpha=1) 

In [15]:
# 모델 학습
model.fit(x_train, y_train) # 학습용 데이터만 사용

Lasso(alpha=1)

In [16]:
# 모델 평가 (R Squared)
print('학습 데이터 성능 :', model.score(x_train, y_train))
print('평가 데이터 성능 :', model.score(x_test, y_test))

# 마찬가지로 과소적합

학습 데이터 성능 : 0.41412544493966097
평가 데이터 성능 : 0.27817828862078753


In [17]:
# 사용된 계수 개수 조회
print('전체 계수 :', len(model.coef_))
print('사용된 계수 : ', len(model.coef_[model.coef_ != 0]))

# w값이 0이 아닌 것만 뽑아보니 2개만 나옴. 285개가 W=0이 되어서 무시된 것!

전체 계수 : 285
사용된 계수 :  2


In [18]:
# 규제 강도 완화 
model = Lasso(alpha=0.1) 
model.fit(x_train, y_train) 
print('학습 데이터 성능 :', model.score(x_train, y_train))
print('평가 데이터 성능 :', model.score(x_test, y_test))

학습 데이터 성능 : 0.5482904661846931
평가 데이터 성능 : 0.35500225702888577


In [19]:
# 사용된 계수 개수 조회
print('전체 계수 :', len(model.coef_))
print('사용된 계수 : ', len(model.coef_[model.coef_ != 0]))

# 강도를 완화할 수록 사용된 계수가 많아진다.

전체 계수 : 285
사용된 계수 :  7


## 제출

제출을 위해 새로 불러온 `diabetes.csv` 데이터를 통해 Ridge 모델을 만들고 학습한 이후 학습 데이터의 score와 테스트 데이터의 score를 아래와 같은 dictionary로 만들어 `result`에 저장하세요.
- `{"train": 0.123, "test": 0.456}`
- Ridge 모델의 alpha 값은 1로 설정합니다.
- Ridge 모델을 생성할 때 반드시 `random_state=SEED`를 추가해야 합니다. 이를 추가하지 않으면 제대로 채점되지 않습니다.

In [20]:
df = pd.read_csv('/mnt/elice/dataset/diabetes.csv')
X = df.iloc[:, :-1]
Y = df.iloc[:, -1]

# SEED 값 바꾸면 제대로 채점되지 않을 수 있습니다.
SEED = 2023

P = PolynomialFeatures(degree=3, include_bias=False)
X_Poly = P.fit_transform(X)
x_train, x_test, y_train, y_test = train_test_split(X_Poly, Y, random_state=SEED)

# TODO: Ridge 모델을 학습하세요.
model = Ridge(alpha = 1)
model.fit(x_train, y_train)

result = {
    "train": model.score(x_train, y_train),
    "test": model.score(x_test, y_test),
}

## 채점 수행

아래 코드는 채점 수행을 위한 코드입니다.

따라서 이를 수정했을시 **채점이 제대로 이루어지지 않습니다.**

**주의**: 채점 코드를 실행하기 전에 반드시 코드 파일을 한번 저장하시길 바랍니다.

In [21]:
import os
import json

assert isinstance(result, dict), \
    "'result' 변수에 요구사항에 맞는 dictionary가 저장되어 있는지 확인하세요."

with open('result.json', 'w') as f:
    json.dump(result, f)

os.system('elice_grade result.json cds_ai_exercise.ipynb')

send files ['result.json', 'cds_ai_exercise.ipynb'] for grade...
waiting result...
waiting result...
done!

Score: 100.000000
Duration: 1.566 seconds
=== Message ===
제출 완료되었습니다.


0